F1 Cost Cap SCM Analysis - Data Preparation
============================================

This notebook prepares F1 constructor championship data for Synthetic Control Method analysis.

Outputs:
- annual_constructor_points.csv (for SCM analysis)
- race_level_points.csv (for time series robustness checks)

Author: Tomasz Solis<br>
Date: November 2025

# ============================================================================
# SETUP
# ============================================================================



In [1]:
import sys
from pathlib import Path

# Ensure src/ is importable
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import fastf1 as ff1
from fastf1.ergast import Ergast

# Project imports
from src.team_name_mapping import (
    normalize_team_column,
    filter_scm_data,
    DEFUNCT_TEAMS
)

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Setup complete")
print(f"📁 Project root: {project_root}")


✅ Setup complete
📁 Project root: /Users/tomasz.solis/repos/private-projects/formula1-cost-cap-scm


# ============================================================================
# LOAD RAW DATA
# ============================================================================


In [2]:
ergast = Ergast()

all_years = range(2014, 2025)
all_standings = []

for year in all_years:
    cs = ergast.get_constructor_standings(season=year)
    temp_df = cs.content[-1].copy()
    temp_df['season'] = year
    all_standings.append(temp_df)

df = pd.concat(all_standings, ignore_index=True)

req         WARNING 	DEFAULT CACHE ENABLED! (237.98 MB) /Users/tomasz.solis/Library/Caches/fastf1


In [3]:
print(f"\n📊 Raw data loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"Seasons: {sorted(df['season'].unique())}")
print(f"Teams (before normalization): {len(df['constructorName'].unique())}")



📊 Raw data loaded: 112 rows, 9 columns
Seasons: [np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
Teams (before normalization): 20


# ============================================================================
# NORMALIZE TEAM NAMES
# ============================================================================

In [4]:
df = normalize_team_column(df, col='constructorName')

print(f"\n✅ Team names normalized")
print(f"Teams (after normalization): {sorted(df['constructorName'].unique())}")


✅ Team names normalized
Teams (after normalization): ['ALPINE', 'ASTON MARTIN', 'CATERHAM', 'FERRARI', 'HAAS', 'MANOR MARUSSIA', 'MCLAREN', 'MERCEDES', 'RB', 'RED BULL', 'SAUBER', 'WILLIAMS']


# ============================================================================
# CREATE ANNUAL DATASET (FOR SCM)
# ============================================================================


In [5]:
# Aggregate to season-level constructor points
annual_df = (
    df.groupby(['season', 'constructorName'])
    .agg({
        'points': 'sum',
        'wins': 'sum',
        'position': 'first'  # Final season position
    })
    .reset_index()
    .sort_values(['season', 'position'])
)

print(f"\n📊 Annual dataset created: {annual_df.shape}")


📊 Annual dataset created: (112, 5)


# ============================================================================
# FILTER TO SCM ANALYSIS PERIOD (2017-2024)
# ============================================================================


In [6]:
# Keep only 2017-2024
scm_df = annual_df[
    (annual_df['season'] >= 2017) & 
    (annual_df['season'] <= 2024)
].copy()

# Exclude defunct teams
defunct_teams = list(DEFUNCT_TEAMS.keys())
scm_df = scm_df[~scm_df['constructorName'].isin(defunct_teams)]

print(f"\n✅ Filtered to SCM period (2017-2024)")
print(f"Shape: {scm_df.shape}")
print(f"Teams: {sorted(scm_df['constructorName'].unique())}")


✅ Filtered to SCM period (2017-2024)
Shape: (80, 5)
Teams: ['ALPINE', 'ASTON MARTIN', 'FERRARI', 'HAAS', 'MCLAREN', 'MERCEDES', 'RB', 'RED BULL', 'SAUBER', 'WILLIAMS']


# ============================================================================
# DATA QUALITY CHECKS
# ============================================================================


In [7]:
print("\n" + "="*80)
print("DATA QUALITY CHECKS")
print("="*80)

# Check 1: All teams present in all years?
team_years = scm_df.groupby('constructorName')['season'].nunique()
print("\n1. Years per team (should be 8 for all):")
print(team_years.sort_values())

incomplete_teams = team_years[team_years < 8]
if len(incomplete_teams) > 0:
    print(f"\n⚠️  WARNING: {len(incomplete_teams)} teams with incomplete data:")
    for team, years in incomplete_teams.items():
        missing_years = set(range(2017, 2025)) - set(scm_df[scm_df['constructorName'] == team]['season'])
        print(f"   {team}: {years} years (missing {sorted(missing_years)})")

# Check 2: Missing values?
print("\n2. Missing values:")
print(scm_df.isnull().sum())

# Check 3: McLaren data specifically
print("\n3. McLaren trajectory (2017-2024):")
mclaren = scm_df[scm_df['constructorName'] == 'MCLAREN'][
    ['season', 'position', 'points', 'wins']
].sort_values('season')
print(mclaren.to_string(index=False))

# Check 4: Pre-treatment period (2017-2020)
print("\n4. Pre-treatment period (2017-2020) - Constructor Points:")
pre_treatment = scm_df[
    (scm_df['season'] >= 2017) & 
    (scm_df['season'] <= 2020)
]
pivot = pre_treatment.pivot_table(
    index='constructorName',
    columns='season',
    values='points',
    fill_value=0
)
print(pivot)


DATA QUALITY CHECKS

1. Years per team (should be 8 for all):
constructorName
ALPINE          8
ASTON MARTIN    8
FERRARI         8
HAAS            8
MCLAREN         8
MERCEDES        8
RB              8
RED BULL        8
SAUBER          8
WILLIAMS        8
Name: season, dtype: int64

2. Missing values:
season             0
constructorName    0
points             0
wins               0
position           0
dtype: int64

3. McLaren trajectory (2017-2024):
 season  position  points  wins
   2017         9    30.0     0
   2018         7    62.0     0
   2019         4   145.0     0
   2020         3   202.0     0
   2021         4   275.0     1
   2022         5   159.0     0
   2023         4   302.0     0
   2024         1   666.0     6

4. Pre-treatment period (2017-2020) - Constructor Points:
season            2017   2018   2019   2020
constructorName                            
ALPINE            57.0  122.0   91.0  181.0
ASTON MARTIN     187.0  111.0   73.0  195.0
FERRARI          

# ============================================================================
# CREATE DONOR POOL DATASETS
# ============================================================================


In [8]:
# Define donor pools
MIDFIELD_TEAMS = ['ALPINE', 'SAUBER', 'HAAS']  # Main donor pool (excludes RB)
MIDFIELD_TEAMS_ROBUST = ['ALPINE', 'SAUBER', 'HAAS', 'RB']  # Robustness check
TOP_3 = ['MERCEDES', 'RED BULL', 'FERRARI']
UNSTABLE = ['ASTON MARTIN', 'WILLIAMS']  # Major ownership changes

# Main donor pool (excludes RB, excludes top 3, excludes unstable)
donor_main = scm_df[scm_df['constructorName'].isin(MIDFIELD_TEAMS)].copy()
donor_main['donor_pool'] = 'main'

# Robustness donor pool (includes RB)
donor_robust = scm_df[scm_df['constructorName'].isin(MIDFIELD_TEAMS_ROBUST)].copy()
donor_robust['donor_pool'] = 'robust'

# Treated unit (McLaren)
treated = scm_df[scm_df['constructorName'] == 'MCLAREN'].copy()
treated['donor_pool'] = 'treated'

# Placebo units
placebo_sauber = scm_df[scm_df['constructorName'] == 'SAUBER'].copy()
placebo_sauber['donor_pool'] = 'placebo_sauber'

placebo_haas = scm_df[scm_df['constructorName'] == 'HAAS'].copy()
placebo_haas['donor_pool'] = 'placebo_haas'

print("\n✅ Donor pools created:")
print(f"   Main donor pool: {MIDFIELD_TEAMS}")
print(f"   Robustness donor pool: {MIDFIELD_TEAMS_ROBUST}")
print(f"   Treated unit: MCLAREN")
print(f"   Placebos: SAUBER, HAAS")


✅ Donor pools created:
   Main donor pool: ['ALPINE', 'SAUBER', 'HAAS']
   Robustness donor pool: ['ALPINE', 'SAUBER', 'HAAS', 'RB']
   Treated unit: MCLAREN
   Placebos: SAUBER, HAAS


# ============================================================================
# SAVE PROCESSED DATA
# ============================================================================


In [9]:
output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

# Save full annual dataset
annual_path = output_dir / "annual_constructor_points.csv"
scm_df.to_csv(annual_path, index=False)
print(f"\n💾 Saved: {annual_path}")

# Save donor pool datasets
donor_main_path = output_dir / "donor_pool_main.csv"
donor_main.to_csv(donor_main_path, index=False)
print(f"💾 Saved: {donor_main_path}")

donor_robust_path = output_dir / "donor_pool_robust.csv"
donor_robust.to_csv(donor_robust_path, index=False)
print(f"💾 Saved: {donor_robust_path}")

# Save treated and placebo units
treated_path = output_dir / "treated_mclaren.csv"
treated.to_csv(treated_path, index=False)
print(f"💾 Saved: {treated_path}")

placebo_sauber_path = output_dir / "placebo_sauber.csv"
placebo_sauber.to_csv(placebo_sauber_path, index=False)
print(f"💾 Saved: {placebo_sauber_path}")

placebo_haas_path = output_dir / "placebo_haas.csv"
placebo_haas.to_csv(placebo_haas_path, index=False)
print(f"💾 Saved: {placebo_haas_path}")


💾 Saved: ../data/processed/annual_constructor_points.csv
💾 Saved: ../data/processed/donor_pool_main.csv
💾 Saved: ../data/processed/donor_pool_robust.csv
💾 Saved: ../data/processed/treated_mclaren.csv
💾 Saved: ../data/processed/placebo_sauber.csv
💾 Saved: ../data/processed/placebo_haas.csv


# ============================================================================
# RACE-LEVEL DATA
# ============================================================================


In [10]:
all_years_data = []

for season in all_years:
    race_results = ergast.get_race_results(season=season)

    season_rows = []
    for rnd, df in enumerate(race_results.content, start=1):
        d = df.groupby(['constructorId', 'constructorName'], as_index=False)['points'].sum()
        d['round'] = rnd
        d['season'] = season
        season_rows.append(d)

    season_df = pd.concat(season_rows, ignore_index=True)

    # cumulative
    season_df['cumulative_points'] = (
        season_df
        .sort_values(['constructorId', 'round'])
        .groupby('constructorId')['points']
        .cumsum()
    )

    all_years_data.append(season_df)

full_df = pd.concat(all_years_data, ignore_index=True)

In [11]:
# Check if we have race-level data (round column exists)
if 'round' in full_df.columns and 'cumulative_points' in full_df.columns:
    print("\n" + "="*80)
    print("RACE-LEVEL DATA PROCESSING")
    print("="*80)
    
    # Filter to SCM period
    race_df = full_df[
        (full_df['season'] >= 2017) & 
        (full_df['season'] <= 2024)
    ].copy()
    
    # Exclude defunct teams
    race_df = race_df[~race_df['constructorName'].isin(defunct_teams)]
    
    # Save race-level data
    race_path = output_dir / "race_level_points.csv"
    race_df.to_csv(race_path, index=False)
    print(f"\n💾 Saved: {race_path}")
    print(f"   Shape: {race_df.shape}")
    print(f"   Columns: {race_df.columns.tolist()}")
else:
    print("\n⚠️  No race-level data available (missing 'round' or 'cumulative_points' columns)")


RACE-LEVEL DATA PROCESSING

💾 Saved: ../data/processed/race_level_points.csv
   Shape: (133, 6)
   Columns: ['constructorId', 'constructorName', 'points', 'round', 'season', 'cumulative_points']


# ============================================================================
# SUMMARY STATISTICS
# ============================================================================


In [12]:
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)

# Pre-treatment vs post-treatment comparison
print("\n📊 Pre-Treatment (2017-2020) vs Treatment (2022-2024) - McLaren:")
mclaren_pre = scm_df[
    (scm_df['constructorName'] == 'MCLAREN') & 
    (scm_df['season'] >= 2017) & 
    (scm_df['season'] <= 2020)
]['points']

mclaren_post = scm_df[
    (scm_df['constructorName'] == 'MCLAREN') & 
    (scm_df['season'] >= 2022) & 
    (scm_df['season'] <= 2024)
]['points']

print(f"Pre-treatment mean: {mclaren_pre.mean():.1f} points")
print(f"Treatment mean: {mclaren_post.mean():.1f} points")
print(f"Difference: {mclaren_post.mean() - mclaren_pre.mean():.1f} points")
print(f"% increase: {((mclaren_post.mean() / mclaren_pre.mean()) - 1) * 100:.1f}%")

# Donor pool pre-treatment means
print("\n📊 Donor Pool Pre-Treatment Means (2017-2020):")
donor_pre = scm_df[
    (scm_df['constructorName'].isin(MIDFIELD_TEAMS)) & 
    (scm_df['season'] >= 2017) & 
    (scm_df['season'] <= 2020)
]
donor_means = donor_pre.groupby('constructorName')['points'].mean().sort_values(ascending=False)
print(donor_means)

print("\n" + "="*80)
print("DATA PREPARATION COMPLETE ✅")
print("="*80)
print(f"\n📁 Processed files saved to: {output_dir}")


SUMMARY STATISTICS

📊 Pre-Treatment (2017-2020) vs Treatment (2022-2024) - McLaren:
Pre-treatment mean: 109.8 points
Treatment mean: 375.7 points
Difference: 265.9 points
% increase: 242.3%

📊 Donor Pool Pre-Treatment Means (2017-2020):
constructorName
ALPINE    112.75
HAAS       42.75
SAUBER     29.50
Name: points, dtype: float64

DATA PREPARATION COMPLETE ✅

📁 Processed files saved to: ../data/processed
